In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from datetime import datetime
import time
import datetime

import eikon as ek
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
ek.set_app_key('d505f265d9bd4c0cb7df44481ece31a0ad8d1974')

Import Data from Eikon API:

In [2]:
Makro_Data = ['USUNR=ECI',  #US Unemploymet
              'USCPI=ECI',  #US CPI
              'USPMI=ECI',  #US PMI
              'EUUNR=ECI',  #EU Unemployment
              'EUHICY=ECI', #EU CPI
              'EUPMI=ECI',  #EU PMI            
              'CHJOB=ECI',  #CH Unemployment
              'CHCPIY=ECI', #CH CPI
              'CHPMI=ECI',  #CH PMI
              'CNURUA=ECI', #China Unemployment  
              'CNCPI=ECI',  #China CPI
              'CNPMIB=ECI', #China PMI
              'RUUNR=ECI',  #Russia Unemployment
              'RUCPIY=ECI', #Russia CPI
              'RUPMIM=ECI', #Russia PMI          
             ]  

Futures = ['CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]

In [3]:
macro_raw = ek.get_timeseries(Makro_Data,
                          start_date='2011-12-31',
                          end_date='2022-01-01',
                          fields='VALUE',
                          interval='monthly')  

macro_raw.columns = macro_raw.columns.str.replace("=", "_")
macro_raw = macro_raw.unstack()
macro_raw = macro_raw.reset_index()
macro_raw = macro_raw.rename(columns={'VALUE': 'Macro_Variable', 0: 'Value'})
macro_raw['Date'] = pd.to_datetime(macro_raw['Date']).dt.date
liste_SQL_macro = macro_raw.values.tolist()
macro_raw.head(5)

,Macro_Variable,Date,Value
0,USUNR_ECI,2011-12-31,8.5
1,USUNR_ECI,2012-01-31,8.3
2,USUNR_ECI,2012-02-29,8.3
3,USUNR_ECI,2012-03-31,8.2
4,USUNR_ECI,2012-04-30,8.2


In [8]:
macro_raw['Type'] = macro_raw['Macro_Variable'].loc[macro_raw['Macro_Variable'] == 'CHCPIY_ECI'] = 'Diff'
macro_raw

C:\Users\lione\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Macro_Variable,Date,Value,Type
0,USUNR_ECI,2011-12-31,8.5,Diff
1,USUNR_ECI,2012-01-31,8.3,Diff
2,USUNR_ECI,2012-02-29,8.3,Diff
3,USUNR_ECI,2012-03-31,8.2,Diff
4,USUNR_ECI,2012-04-30,8.2,Diff
...,...,...,...,...
1810,RUPMIM_ECI,2021-08-31,46.5,Diff
1811,RUPMIM_ECI,2021-09-30,49.8,Diff
1812,RUPMIM_ECI,2021-10-31,51.6,Diff
1813,RUPMIM_ECI,2021-11-30,51.7,Diff


In [ ]:
xx = pd.DataFrame()
xx['CHCPIY_DIFF'] = macro['CHCPIY_ECI']-macro['CHCPIY_ECI'].shift(1)
xx['CHJOB_DIFF'] = macro['CHJOB_ECI']-macro['CHJOB_ECI'].shift(1)
xx['CHPMI_LOG'] = np.log(macro['CHPMI_ECI']) - np.log(macro['CHPMI_ECI'].shift(1))
xx['CNCPI_DIFF'] = macro['CNCPI_ECI']-macro['CNCPI_ECI'].shift(1)
xx['CNPMIB_LOG'] = np.log(macro['CNPMIB_ECI']) - np.log(macro['CNPMIB_ECI'].shift(1))
xx['EUHICY_DIFF'] = macro['EUHICY_ECI']-macro['EUHICY_ECI'].shift(1)
xx['EUUNR_DIFF'] = macro['EUUNR_ECI']-macro['EUUNR_ECI'].shift(1)
xx['RUCPIY_DIFF'] = macro['RUCPIY_ECI']-macro['RUCPIY_ECI'].shift(1)
xx['RUUNR_DIFF'] = macro['RUUNR_ECI']-macro['RUUNR_ECI'].shift(1)
xx['USPMI_LOG'] = np.log(macro['USPMI_ECI']) - np.log(macro['USPMI_ECI'].shift(1))
xx['USUNR_DIFF'] = macro['USUNR_ECI']-macro['USUNR_ECI'].shift(1)
xx['USCPI_DIFF'] = macro['USCPI_ECI']
xx = xx.dropna()
xx

In [4]:
futures_raw = pd.DataFrame()
try:
    itemName=Futures
    dateStart="2011-12-31"
    dateEnd="2022-01-01"

    #Create Loop to inport all Futures:
    for y in range(0,len(Futures)):
        data_input = ek.get_timeseries(Futures[y],fields=["Close"], start_date = dateStart , 
                                    end_date = dateEnd , interval='monthly', corax = 'adjusted' )
        data_input['Instrument'] = Futures[y]
        futures_raw = futures_raw.append(data_input)
except ek.EikonError as ex:
    print('Eikon Error')
    print(ex)
except Exception as ex:
    print(ex)

futures_raw = futures_raw.sort_values(['Instrument','Date'])
futures_raw["Return"] = futures_raw.groupby("Instrument")['CLOSE'].apply(lambda x: np.log(x) - np.log(x.shift(1)))
futures_raw = futures_raw.reset_index()
futures_raw['Date'] = pd.to_datetime(futures_raw['Date']).dt.date
liste_SQL_futures = futures_raw.values.tolist()
futures_raw

OJc1,Date,CLOSE,Instrument,Return
0,2011-12-31,98.83,CLc1,NaN
1,2012-01-31,98.48,CLc1,-0.003548
2,2012-02-29,107.07,CLc1,0.083629
3,2012-03-31,103.02,CLc1,-0.038560
4,2012-04-30,104.87,CLc1,0.017798
...,...,...,...,...
1326,2021-08-31,706.75,Wc1,0.004254
1327,2021-09-30,725.50,Wc1,0.026184
1328,2021-10-31,772.75,Wc1,0.063095
1329,2021-11-30,773.75,Wc1,0.001293


In [ ]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

In [ ]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS MACRO_RAW;''')
c.execute('''DROP TABLE IF EXISTS FUTURES_RAW;''')

In [ ]:
c.execute('''CREATE TABLE MACRO_RAW (
    "Macro_Variable" TEXT,
    DATE DATETIME,
	"Value" NUMERIC
);''')

c.execute('''CREATE TABLE FUTURES_RAW (
    DATE DATETIME,
	"CLOSE" NUMERIC,
    "Instrument" TEXT,
    "Return" NUMERIC
);''')

In [ ]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
c = conn.cursor()
c.executemany("INSERT INTO MACRO_RAW(Macro_Variable, DATE, Value) VALUES (?,?,?)", liste_SQL_macro)    
conn.commit()

c.executemany("INSERT INTO FUTURES_RAW(DATE, CLOSE, Instrument, Return) VALUES (?,?,?,?)", liste_SQL_futures)    
conn.commit()

In [ ]:
#Select Statement to import the SQL Data back into Python:   
c = conn.cursor()
c.execute("SELECT * FROM MACRO_RAW")
macro_sql_data=c.fetchall()
macro_raw_sql = pd.DataFrame(macro_sql_data,columns=['Macro_Variable', 'DATE', 'Value'])
macro_raw_sql = macro_raw_sql.set_index('DATE')
macro_raw_sql

#Select Futures Data from SQL DB:   
c.execute("SELECT * FROM FUTURES_RAW")
futures_sql_data=c.fetchall()
futures_raw_sql = pd.DataFrame(futures_sql_data,columns=['DATE', 'CLOSE', 'Instrument', 'Return'])
futures_raw_sql = futures_raw_sql.set_index('DATE')
futures_raw_sql

In [ ]:
macro_raw_pivot = macro_raw_sql.pivot(columns='Macro_Variable', values=['Value'])
macro_raw_pivot = macro_raw_pivot.droplevel(level=0, axis=1)
macro_raw_pivot.to_excel('Files\Macro_raw_pivot.xlsx')
macro_raw_pivot

futures_raw_close_pivot = futures_raw_sql.pivot(columns='Instrument', values=['CLOSE'])
futures_raw_close_pivot = futures_raw_close_pivot.droplevel(level=0, axis=1)
futures_raw_close_pivot.to_excel('Files\Futures_raw_CLOSE_pivot.xlsx')
futures_raw_close_pivot

futures_raw_returns_pivot = futures_raw_sql.pivot(columns='Instrument', values=['Return'])
futures_raw_returns_pivot = futures_raw_returns_pivot.droplevel(level=0, axis=1)
futures_raw_returns_pivot.to_excel('Files\Futures_raw_returns_pivot.xlsx')
futures_raw_returns_pivot